In [1209]:
import streamlit as st
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [1210]:
df=pd.read_csv('info.csv')

In [1211]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [1212]:
df.shape

(63888, 9)

### So this is that dataset we have. We need to create some columns and extract few to get the desired data. Eventually we want our data to have columns:

- batting team
- bowling team
- city
- current_score
- balls left
- wickets_left
- current_run_rate
- last five

### Now we already have few columns as we want it in our dataset. Batting team and bowling team data we already have. we also have city column but it has some null values which we need to figure out how to handle that. For rest all we need do some manipulation.

### Now we will start our feature extraction from the city column. To fill the empty values we will use venue column.



In [1213]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

### Here we are checking the values in ‘venue’ column where city column has null values. If we notice carefully the first word in venue is actually the name of the city where the venue exists for e.g. Dubai in Dubai International Cricket Stadium or Melbourne in Melbourne Cricket Ground.



In [1214]:
cities=np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])


In [1215]:
df['city']=cities


In [1216]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

### So we store all the first word of venue column in variable named cities and then use it to fill the the city column. Now there are no null values in our dataset. But still there is one more thing left. Our dataset is a ball-by-ball dataset which means if there are 63000 rows that means that many balls have been bowled and played.

In [1217]:
df.drop(columns=['Unnamed: 0','venue'],inplace=True)

In [1218]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo


In [1219]:
eligible_cities=df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

### This shows that there are certain cities where very few deliveries have been played. So we can ignore those cities and only consider the ones which have at least 600 deliveries.

In [1220]:
df=df[df['city'].isin(eligible_cities)]

In [1221]:
print(df['runs'].dtype)
df['runs'] = pd.to_numeric(df['runs'], errors='coerce')

int64


In [1222]:
df['runs'].fillna(0, inplace=True)  # Example of filling with 0


In [1223]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()


### Now our city column is complete. Coming to current_runs column which is very easy to extract from the runs column. A simple cumsum() function (used to find the cumulative sum of a column) will do the work for us.



In [1224]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3
...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127


### Now our next target is to create a ‘balls_left’ column for which firstly we would be creating to new columns: ‘overs’ and ‘balls’ which tells us how many overs have been completed and how many balls of the current over has been bowled respectively. The code is very simple for that.

In [1225]:
df['over']=df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no']=df['ball'].apply(lambda x:str(x).split(".")[1])

In [1226]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5
...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6


### Now by using a simple formula we can create a ‘balls_bowled’ column that is how many balls have been bowled. Formula would be

``` balls_bowled = (overs * 6) + balls```

In [1227]:
df['balls_bowled']=(df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [1228]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,117
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,118
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,119
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,120


### And now finally we can create our desired column ‘balls_left’ by subtracting balls_bowled from 120 because there are total 120 balls in an innings. sometimes because of extras (wide, no ball …) the ball count exceeds 120 so in such case we can simply give the value of 0.

In [1229]:
df['balls_left']=120-df['balls_bowled']
df['balls_left']=df['balls_left'].apply(lambda x:0 if x<0 else x)

In [1230]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,117,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,118,2
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,119,1
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,120,0


### Now if we look at the ‘player_dismissed’ column it has either value 0 or name of the player got out at that particular ball. First we will replace all the names with 1 and then apply the cumsum() function on it so we can get the total wickets gone and we will subtract it from 10 to get the ‘wickets_left’ column.



In [1231]:
print(df['player_dismissed'].dtype)


object


In [1232]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [1233]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1


In [1234]:
df['crr']=(df['current_score']*6)/df['balls_bowled']

In [1235]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000


### Now we need a column that has total runs scored in last five overs. Obviously we will have null values in this column for first 5 overs.



In [1236]:

df['batting_team'] = df['batting_team'].astype('category')
df['bowling_team'] = df['bowling_team'].astype('category')
df['city'] = df['city'].astype('category')
df['over'] = pd.to_numeric(df['over'], errors='coerce')
df['ball_no'] = pd.to_numeric(df['ball_no'], errors='coerce')
print(df.dtypes)

match_id               int64
batting_team        category
bowling_team        category
ball                 float64
runs                   int64
player_dismissed       int64
city                category
current_score          int64
over                   int64
ball_no                int64
balls_bowled           int64
balls_left             int64
wickets_left           int64
crr                  float64
dtype: object


In [1237]:
#df['batting_team'].fillna(0, inplace=True)  # Replace NaNs with 0, or use another method
#df['bowling_team'].fillna(0, inplace=True)  # Replace NaNs with 0, or use another method
#df['city'].fillna(0, inplace=True)  # Replace NaNs with 0, or use another method
#df['over'].fillna(0, inplace=True)  # Replace NaNs with 0, or use another method
#df['ball_no'].fillna(0, inplace=True)  # Replace NaNs with 0, or use another method
#print(df.dtypes)

In [1238]:
groups=df.groupby('match_id')

match_ids=df['match_id'].unique()
last_five=[]
for id in match_ids:
    group = groups.get_group(id)
    group['runs'] = pd.to_numeric(group['runs'], errors='coerce')
    last_five.extend(groups.get_group(id).rolling(window=30)['runs'].sum().fillna(0).values.tolist())

In [1239]:
df['last_five']=last_five

In [1240]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000,0.0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000,0.0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000,0.0
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000,0.0
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256,32.0
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932,32.0
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521,32.0
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000,33.0


### Now we have to create a last column which would be our target column. Total runs scored in that innings.


In [1241]:
final_df=df.groupby('match_id')['runs'].sum().reset_index().merge(df,on='match_id')

In [1242]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [1243]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Australia,Sri Lanka,Melbourne,0,119,10,0.000000,0.0,168
1,Australia,Sri Lanka,Melbourne,0,118,10,0.000000,0.0,168
2,Australia,Sri Lanka,Melbourne,1,117,10,2.000000,0.0,168
3,Australia,Sri Lanka,Melbourne,3,116,10,4.500000,0.0,168
4,Australia,Sri Lanka,Melbourne,3,115,10,3.600000,0.0,168
...,...,...,...,...,...,...,...,...,...
50496,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128
50497,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
50498,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
50499,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128


### Now we will drop all the columns which we dont want to have for our model and keep those which we created just now. Also we will shuffle the data to avoid any kind of bias.

In [1244]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [1245]:
final_df.dropna(inplace=True)

In [1246]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [1247]:
final_df=final_df.sample(final_df.shape[0])

In [1248]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
38593,Sri Lanka,England,London,5,116,10,7.500000,0.0,183
18581,West Indies,South Africa,Johannesburg,59,45,6,4.720000,18.0,131
3998,Pakistan,New Zealand,Auckland,5,116,10,7.500000,0.0,201
44081,Pakistan,Sri Lanka,Colombo,8,112,10,6.000000,0.0,175
10015,Australia,South Africa,Cape Town,150,25,7,9.473684,31.0,193
...,...,...,...,...,...,...,...,...,...
41278,India,Sri Lanka,Mirpur,93,35,8,6.564706,41.0,130
23788,Afghanistan,India,St Lucia,57,50,7,4.885714,26.0,115
15029,Sri Lanka,New Zealand,Auckland,115,13,2,6.448598,49.0,115
21629,Pakistan,South Africa,Nottingham,4,117,10,8.000000,0.0,149


### With this we end out feature extraction part of the project. So after lot of work we finally have the exact required data we wanted at the start.

### So lets now begin with model building process. For that first we will divide our dataset in training set and testing set using train_test_split module of sklearn library



In [1249]:
import sklearn
print(sklearn.__version__)

1.5.1


In [1250]:
#Train-Test-Split

X=final_df.drop(columns=['runs_x'])
y=final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [1251]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
29201,South Africa,Australia,Cape Town,65,58,8,6.290323,30.0
40104,Bangladesh,India,Mirpur,27,84,7,4.500000,14.0
13514,New Zealand,West Indies,Mount Maunganui,170,22,8,10.408163,63.0
41760,Bangladesh,Sri Lanka,Chittagong,70,46,4,5.675676,23.0
21827,Sri Lanka,West Indies,London,80,43,7,6.233766,30.0
...,...,...,...,...,...,...,...,...
10223,India,Bangladesh,Delhi,69,60,8,6.900000,40.0
35721,England,New Zealand,Auckland,172,26,7,10.978723,55.0
33665,Sri Lanka,Pakistan,Colombo,87,40,8,6.525000,40.0
931,India,England,Nagpur,79,52,7,6.970588,30.0


In [1252]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
#from xgboost import XGBRegressor

#from sklearn.metrics import accuracy_score


from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [1253]:
teams = ['Australia', 'India', 'Bangladesh', 'New Zealand', 'South Africa', 
         'England', 'West Indies', 'Afghanistan', 'Pakistan', 'Sri Lanka']
cities = ['Colombo', 'Mirpur', 'Johannesburg', 'Dubai', 'Auckland', 
          'Cape Town', 'London', 'Pallekele', 'Barbados', 'Sydney', 
          'Melbourne', 'Durban', 'St Lucia', 'Wellington', 'Lauderhill', 
          'Hamilton', 'Centurion', 'Manchester', 'Abu Dhabi', 'Mumbai', 
          'Nottingham', 'Southampton', 'Mount Maunganui', 'Chittagong', 
          'Kolkata', 'Lahore', 'Delhi', 'Nagpur', 'Chandigarh', 'Adelaide', 
          'Bangalore', 'St Kitts', 'Cardiff', 'Christchurch', 'Trinidad']

In [1254]:
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(categories=[teams, teams, cities],sparse_output=False,drop='first'),['batting_team','bowling_team','city']),
    ('num', StandardScaler(), ['current_score', 'balls_left', 'wickets_left', 'crr', 'last_five'])
])

In [1255]:
#pipe=Pipeline(steps=[
  # ('step1',trf),
    #('step2',StandardScaler()),
 #   ('step3',XGBRegressor(n_estimators=300,learning_rate=0.2,max_depth=12,random_state=1))

#])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

In [1201]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(categories=[['Australia',
                                                                             'India',
                                                                             'Bangladesh',
                                                                             'New '
                                                                             'Zealand',
                                                                             'South '
                                                                             'Africa',
                                                                             'England',
                                                                             'West '
                                                                             'Indies',
                                                                             'Afghanistan',
                                                                             'Pakistan',
                                                                             'Sri '
                                                                             'Lanka'],
                                                                            ['Australia',
                                                                             'India',
                                                                             'Bangladesh',
                                                                             'New '
                                                                             'Zealand',
                                                                             'South '
                                                                             'Africa',
                                                                             'England',
                                                                             'West '
                                                                             'Indies',
                                                                             'Afghanistan',
                                                                             'Pakistan',
                                                                             'Sri '
                                                                             'Lan...
                                                                             'Manchester',
                                                                             'Abu '
                                                                             'Dhabi',
                                                                             'Mumbai',
                                                                             'Nottingham',
                                                                             'Southampton',
                                                                             'Mount '
                                                                             'Maunganui',
                                                                             'Chittagong',
                                                                             'Kolkata',
                                                                             'Lahore',
                                                                             'Delhi',
                                                                             'Nagpur',
                                                                             'Chandigarh',
                                                                             'Adelaide', ...]],
                                                                drop='first',
                                                     

In [1202]:
Y_test_pred=pipe.predict(X_test)
Y_train_pred = pipe.predict(X_train)

In [1203]:
print(r2_score(Y_test,Y_test_pred))
print(mean_absolute_error(Y_test,Y_test_pred))

0.9460035994521044
3.317698229233944


In [1204]:
# Mean Absolute Error
mae_train = mean_absolute_error(Y_train, Y_train_pred)
mae_test = mean_absolute_error(Y_test, Y_test_pred)

# Mean Squared Error
mse_train = mean_squared_error(Y_train, Y_train_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

# R-squared (explained variance)
r2_train = r2_score(Y_train, Y_train_pred)
r2_test = r2_score(Y_test, Y_test_pred)

# Print the results
print(f"Train MAE: {mae_train}")
print(f"Test MAE: {mae_test}")

print(f"Train MSE: {mse_train}")
print(f"Test MSE: {mse_test}")

print(f"Train R²: {r2_train}")
print(f"Test R²: {r2_test}")


Train MAE: 1.289237386256483
Test MAE: 3.317698229233944
Train MSE: 10.360004995119082
Test MSE: 56.62904718276939
Train R²: 0.990195861970658
Test R²: 0.9460035994521044


In [1205]:
st.table(df)

DeltaGenerator()

In [1206]:
import pickle

In [1207]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [1208]:
eligible_cities

['Colombo',
 'Mirpur',
 'Johannesburg',
 'Dubai',
 'Auckland',
 'Cape Town',
 'London',
 'Pallekele',
 'Barbados',
 'Sydney',
 'Melbourne',
 'Durban',
 'St Lucia',
 'Wellington',
 'Lauderhill',
 'Hamilton',
 'Centurion',
 'Manchester',
 'Abu Dhabi',
 'Mumbai',
 'Nottingham',
 'Southampton',
 'Mount Maunganui',
 'Chittagong',
 'Kolkata',
 'Lahore',
 'Delhi',
 'Nagpur',
 'Chandigarh',
 'Adelaide',
 'Bangalore',
 'St Kitts',
 'Cardiff',
 'Christchurch',
 'Trinidad']